# Getting all Dispute and Slashing Data
Below is the process used to get dispute and slashing data from The Graph.

In [1]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

In [2]:
# function used to query each type of event from The Graph
def query_theGraph(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}"
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + indexer + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [4]:
# get stakeSlashed data
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

stake_slashed_query = """query{stakeSlasheds () {
    id
    blockNumber
    logIndex
    indexer
    tokens
    reward
    beneficiary
}
}"""
field_name = 'stakeSlasheds'
stake_slashed_df = query_theGraph(stake_slashed_query, field_name, url, False)

# save stakeDeposited data
stake_slashed_df['type'] = 'stakeSlasheds'
stake_slashed_df.to_csv('stakeSlasheds.csv', index = False, header = True)

In [5]:
stake_slashed_df

,id,blockNumber,logIndex,indexer,tokens,reward,beneficiary,type
0,0x1596590788e3b6464685e27ab099daa36f8bf9236153...,13058230,48,0x76e84025978a0c16fc7ba483718b667388f2973c,29400814429489079062405,14700407214744539531202,0xe3671ff90401c21e2bb55b2dfab4506cede113e5,stakeSlasheds
1,0x1596590788e3b6464685e27ab099daa36f8bf9236153...,13058230,53,0x76e84025978a0c16fc7ba483718b667388f2973c,28665794068751852085844,14332897034375926042922,0xe3671ff90401c21e2bb55b2dfab4506cede113e5,stakeSlasheds
2,0x1596590788e3b6464685e27ab099daa36f8bf9236153...,13058230,58,0x76e84025978a0c16fc7ba483718b667388f2973c,27949149217033055783698,13974574608516527891849,0xe3671ff90401c21e2bb55b2dfab4506cede113e5,stakeSlasheds
3,0x1596590788e3b6464685e27ab099daa36f8bf9236153...,13058230,63,0x76e84025978a0c16fc7ba483718b667388f2973c,27250420486607229389106,13625210243303614694553,0xe3671ff90401c21e2bb55b2dfab4506cede113e5,stakeSlasheds
4,0x1596590788e3b6464685e27ab099daa36f8bf9236153...,13058230,68,0x76e84025978a0c16fc7ba483718b667388f2973c,26569159974442048654378,13284579987221024327189,0xe3671ff90401c21e2bb55b2dfab4506cede113e5,stakeSlasheds
5,0x1596590788e3b6464685e27ab099daa36f8bf9236153...,13058230,73,0x76e84025978a0c16fc7ba483718b667388f2973c,25904930975080997438019,12952465487540498719009,0xe3671ff90401c21e2bb55b2dfab4506cede113e5,stakeSlasheds


In [13]:
# function used to query each type of event from The Graph
def query_theGraph_dispute(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:createdAt, where: {createdAt_gt:0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:createdAt, where: {createdAt_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
            print(indexer)
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99 and n > 1:
            break
        
    return pd.DataFrame(records)

In [14]:
#get dispute data
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

dispute_query = '''query{
	disputes(){
    id
    subgraphDeployment{
        id
    }
    fisherman
    {
        id
    }
    deposit
    createdAt
    closedAt
    status
    tokensSlashed
    tokensBurned
    tokensRewarded
    type
    indexer
    {
        id
    }
    allocation
    {
        id
    }
  }
}
'''

field_name = 'disputes'

dispute_df = query_theGraph_dispute(dispute_query, field_name, url, False)

1644451343
noblock


In [15]:
dispute_df

,id,subgraphDeployment,fisherman,deposit,createdAt,closedAt,status,tokensSlashed,tokensBurned,tokensRewarded,type,indexer,attestation,allocation
0,0xfe748e4320d3346a18738b775520aa6a7f0f1d2b31af...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254498,1624924250,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x2a34c9fbd175361f7e0a197f0f1cf0b43653...
1,0xa3770bc7eb1097f6b7a806ee22b7221f7664da3195be...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254826,1624924692,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x4b0ccf7e8e45549588d31b325de24a767fd9...
2,0x6f6b765d937dbb774bbfce23309cd41658be4d39dea3...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254879,1624926091,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x3b1361093cb7b4a98c05a6455e2c76fd5099...
3,0x08e959b7cf82f4935a5063721d7b7a01a0b33ebf0c80...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254993,1624479967,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x3a8dace06b3db1f870ffa9c2d0f50e4f7945...
4,0xeb77920c7a47d56106b11c2ea979116cfa956573578c...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255065,1624921035,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0xdc56c66b653aa668dd068ea438c9c7f6a334...
5,0xa8b6762193318dcd253cbe6426cf335a0b194c8b63e0...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255239,1624926091,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x7ac5df68ed7f0980c91bceac750a7bb94f9c...
6,0xf8ccaf6e7a1e9766ea5d77c24a4b17c84863e97f08f3...,{'id': '0x500a8e47cbdeca7386448ae9e7d52578871b...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255363,1624926091,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0xd3c9dc4a25e33e3c88355d47d1409e15c10e...
7,0x2232bff35186ea2734e36126561196d0815449154a59...,{'id': '0x31edcacc9a53bc8ab4be2eeb0d873409da4c...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255494,1629774915,Draw,0,0,0,Indexing,{'id': '0x85fe868adf7f5950b052469075556fb207e5...,None,{'id': '0xcac11d8445765d0a81e53cef0df09c45b0ec...
8,0x41e44fbcf77dd4c99ccfb254c278d68f385028cbe67b...,{'id': '0x31edcacc9a53bc8ab4be2eeb0d873409da4c...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255589,1629774915,Draw,0,0,0,Indexing,{'id': '0x85fe868adf7f5950b052469075556fb207e5...,None,{'id': '0x249a6fbf13e761e5a465f30afa6eb4184c57...
9,0xa6e48245e348d1533e31cba5bac4da3125104c927c53...,{'id': '0x949d775a90ef63b46d95fc90cf326dea9cce...,{'id': '0xe3671ff90401c21e2bb55b2dfab4506cede1...,10000000000000000000000,1627293099,1629409169,Accepted,29400814429489079062404,14700407214744539531202,14700407214744539531202,Indexing,{'id': '0x76e84025978a0c16fc7ba483718b667388f2...,None,{'id': '0x5037caea050b4dda604ef5efe908715e5cb7...


In [17]:
dispute_df['subgraphDeployment'] = dispute_df['subgraphDeployment'].apply(pd.Series)
dispute_df['fisherman'] = dispute_df['fisherman'].apply(pd.Series)
dispute_df

,id,subgraphDeployment,fisherman,deposit,createdAt,closedAt,status,tokensSlashed,tokensBurned,tokensRewarded,type,indexer,attestation,allocation
0,0xfe748e4320d3346a18738b775520aa6a7f0f1d2b31af...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254498,1624924250,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x2a34c9fbd175361f7e0a197f0f1cf0b43653...
1,0xa3770bc7eb1097f6b7a806ee22b7221f7664da3195be...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254826,1624924692,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x4b0ccf7e8e45549588d31b325de24a767fd9...
2,0x6f6b765d937dbb774bbfce23309cd41658be4d39dea3...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254879,1624926091,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x3b1361093cb7b4a98c05a6455e2c76fd5099...
3,0x08e959b7cf82f4935a5063721d7b7a01a0b33ebf0c80...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620254993,1624479967,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x3a8dace06b3db1f870ffa9c2d0f50e4f7945...
4,0xeb77920c7a47d56106b11c2ea979116cfa956573578c...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255065,1624921035,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0xdc56c66b653aa668dd068ea438c9c7f6a334...
5,0xa8b6762193318dcd253cbe6426cf335a0b194c8b63e0...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255239,1624926091,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0x7ac5df68ed7f0980c91bceac750a7bb94f9c...
6,0xf8ccaf6e7a1e9766ea5d77c24a4b17c84863e97f08f3...,0x500a8e47cbdeca7386448ae9e7d52578871b9942ebbe...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255363,1624926091,Draw,0,0,0,Indexing,{'id': '0x5a8904be09625965d9aec4bffd30d853438a...,None,{'id': '0xd3c9dc4a25e33e3c88355d47d1409e15c10e...
7,0x2232bff35186ea2734e36126561196d0815449154a59...,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255494,1629774915,Draw,0,0,0,Indexing,{'id': '0x85fe868adf7f5950b052469075556fb207e5...,None,{'id': '0xcac11d8445765d0a81e53cef0df09c45b0ec...
8,0x41e44fbcf77dd4c99ccfb254c278d68f385028cbe67b...,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,{'id': '0x992bb240b1ef27bc95a2e4767d9de6f8bf6d...,10000000000000000000000,1620255589,1629774915,Draw,0,0,0,Indexing,{'id': '0x85fe868adf7f5950b052469075556fb207e5...,None,{'id': '0x249a6fbf13e761e5a465f30afa6eb4184c57...
9,0xa6e48245e348d1533e31cba5bac4da3125104c927c53...,0x949d775a90ef63b46d95fc90cf326dea9cceef9b3c4f...,{'id': '0xe3671ff90401c21e2bb55b2dfab4506cede1...,10000000000000000000000,1627293099,1629409169,Accepted,29400814429489079062404,14700407214744539531202,14700407214744539531202,Indexing,{'id': '0x76e84025978a0c16fc7ba483718b667388f2...,None,{'id': '0x5037caea050b4dda604ef5efe908715e5cb7...
